# Getting url

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
data

<Response [200]>

In [3]:
soup = BeautifulSoup(data.content,"html.parser")

# Article Fetching

In [4]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

# Collecting all urls

In [5]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

In [6]:
urllist

['https://www.opindia.com/latest-news/page/2/',
 'https://www.opindia.com/latest-news/page/3/',
 'https://www.opindia.com/latest-news/page/4/',
 'https://www.opindia.com/latest-news/page/5/',
 'https://www.opindia.com/latest-news/page/6/',
 'https://www.opindia.com/latest-news/page/7/',
 'https://www.opindia.com/latest-news/page/8/',
 'https://www.opindia.com/latest-news/page/9/',
 'https://www.opindia.com/latest-news/page/10/',
 'https://www.opindia.com/latest-news/page/11/',
 'https://www.opindia.com/latest-news/page/12/',
 'https://www.opindia.com/latest-news/page/13/',
 'https://www.opindia.com/latest-news/page/14/',
 'https://www.opindia.com/latest-news/page/15/',
 'https://www.opindia.com/latest-news/page/16/',
 'https://www.opindia.com/latest-news/page/17/',
 'https://www.opindia.com/latest-news/page/18/',
 'https://www.opindia.com/latest-news/page/19/',
 'https://www.opindia.com/latest-news/page/20/']

# Articles

In [7]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

# Preprocessing

In [8]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

# Creating clusters

In [10]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [11]:
cl_res.labels_

array([4, 1, 1, ..., 0, 0, 0])

In [14]:
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [15]:
Q.head()

,Article,Cluster
0,HERE IS WHI THE HARYANA GOVT CANCEL THE LICEN...,4
1,MUMBAI BMC FLOAT TENDER FOR 128 CRORE URDU CE...,1
2,REALITI OF RAHUL GANDHI AT CAMBRIDG UNIVERS E...,1
3,FORMER PUNJAB CONGRESS CHIEF NAVJOT SIDHU TO ...,0
4,LAND FOR JOB SCAM CBI RAID MULTIPL LOCAT LINK...,0


In [16]:
Q

,Article,Cluster
0,HERE IS WHI THE HARYANA GOVT CANCEL THE LICEN...,4
1,MUMBAI BMC FLOAT TENDER FOR 128 CRORE URDU CE...,1
2,REALITI OF RAHUL GANDHI AT CAMBRIDG UNIVERS E...,1
3,FORMER PUNJAB CONGRESS CHIEF NAVJOT SIDHU TO ...,0
4,LAND FOR JOB SCAM CBI RAID MULTIPL LOCAT LINK...,0
...,...,...
1971,JAHANGIRPURI RIOT POLIC CRIME BRANCH BEGIN PR...,0
1972,PERMISS TO BE TAKEN FOR PLAY HANUMAN CHALISA ...,0
1973,MADHYA PRADESH GOVT TO REHABILIT HASEENA FAKH...,0
1974,BJP PRESID JP NADDA BLAME THE OPPOSIT FOR FUE...,0


In [17]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [18]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

# News recommendation by Categorywise

In [19]:
Q

,Article,Cluster,category
0,HERE IS WHI THE HARYANA GOVT CANCEL THE LICEN...,4,entertainment
1,MUMBAI BMC FLOAT TENDER FOR 128 CRORE URDU CE...,1,politics
2,REALITI OF RAHUL GANDHI AT CAMBRIDG UNIVERS E...,1,politics
3,FORMER PUNJAB CONGRESS CHIEF NAVJOT SIDHU TO ...,0,geopolitics
4,LAND FOR JOB SCAM CBI RAID MULTIPL LOCAT LINK...,0,geopolitics
...,...,...,...
1971,JAHANGIRPURI RIOT POLIC CRIME BRANCH BEGIN PR...,0,geopolitics
1972,PERMISS TO BE TAKEN FOR PLAY HANUMAN CHALISA ...,0,geopolitics
1973,MADHYA PRADESH GOVT TO REHABILIT HASEENA FAKH...,0,geopolitics
1974,BJP PRESID JP NADDA BLAME THE OPPOSIT FOR FUE...,0,geopolitics
